# Electric Field Calculations - 1

M&I 4e

Ch 13 Electric Field

- §13.X &ndash; Student Resources

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]*
>
>   
>
>   In the region shown in the diagram above, there is an electric field due to
>   a point charge located at the center of the magenta circle. The orange arrows
>   indicate the magnitude and direction of the electric field at the locations
>   shown. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='1a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the sign of the source charge?__
>
>   (select one:)
>   
>   - [ ] &nbsp; positive
>   - [ ] &nbsp; negative
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] positive


<h3 id='1b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(B)
>
>   Now, a particle whose charge is $-8 \times 10^{-9}$ [C] is placed at location
>   $A$.
>
>   __Which arrow ($a$-$j$) best indicates the direction of the electric force
>   on the $-8 \times 10^{-9}$ [C] charge?
>
>   
>
>   (select one:)
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] f

<h3 id='1c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(C)
>
>   The electric field at location $A$ has the value $\langle 5000, 5000, 0\rangle$ [N/C].
>
>   __What is the unit vector in the direction of $\vec{E}$ at this location?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; 
>   $\hat{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?} \right\rangle$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the electric force on the $-8 \times 10^{-9}$ charge?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; 
>   $
>       \vec{F} = \left\langle\ 
>           \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ 
>       \right\rangle\ 
>       \left[\mathrm{N}\right]
>   $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the unit vector in the direction of this electric force?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; 
>   $
>       \hat{F} = \left\langle\ 
>           \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ 
>       \right\rangle\ 
>   $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]*
>
>   A proton is observed to have an instantaneous acceleration of $9 \times 10^{11}$ [m/s²].
>
>   __What is the magnitude $E$ of the electric field at the proton's location?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{N/C}\right]$
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]*
>
>   In a hydrogen atom in its ground state, the electron is on average a distance
>   of about $0.5 \times 10^{-10}$ [m] from the proton. 
>
>   __What is the magnitude of the electric field due to the proton, at this distance from the proton?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{N/C}\right]$
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

---